* this notebook contains the main takeaway, multihead-attention implementation 

In [1]:
!pip install torch

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+1+max_length]
            self.input_ids.append(input_chunk)
            self.target_ids.append(target_chunk)

    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
    

def create_dataloader(text, batch_size=4, max_length = 256, stride=128, shuffle=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    datalaoder = DataLoader(dataset, batch_size=batch_size, shuffle = shuffle)
    return datalaoder



with open("C:\\Users\\Mukund Agarwalla\\Desktop\\NLP\\Attention Mechanism\\text.txt", "r" , encoding="utf8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

vocab_size = 50257
output_dim = 256
max_len = 1024
context_length = 1024

token_embedding_layer = nn.Embedding(vocab_size, output_dim)
pos_embedding_kayer = nn.Embedding(context_length, output_dim)

max_length = 4
dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=max_length)

In [8]:
for batch in dataloader:
    x,y = batch

    token_embedding = token_embedding_layer(x)
    pos_embedding = pos_embedding_kayer(torch.arange(max_length))

    input_embeddings = token_embedding + pos_embedding

    break

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list